In [1]:
import numpy as np
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets
import plotly.graph_objs as go
from ipywidgets import interact

npz_dict = [
    '../output/camera/leaving-pr-57/output_3D/output_keypoints_3d.npz',
    '../output/camera/leaving-pr-55/output_3D/output_keypoints_3d.npz',
    '../output/camera/leaving-pr-54/output_3D/output_keypoints_3d.npz',
    '../output/camera/leaving-pr-53/output_3D/output_keypoints_3d.npz',
    '../output/camera/leaving-pr-52/output_3D/output_keypoints_3d.npz',
    '../output/camera/leaving-pr-51/output_3D/output_keypoints_3d.npz',
]

all_data = {}

for index, npz in enumerate(npz_dict):      # Load the data from the provided .npz file
    data = np.load(npz)
    print(data.files)  # Inspect the keys in the loaded data
    # Inspect the shape and contents of the 'reconstruction' dataset
    reconstruction_data = data['reconstruction']
    reconstruction_data.shape
    all_data[index] = reconstruction_data



['reconstruction']
['reconstruction']
['reconstruction']
['reconstruction']
['reconstruction']
['reconstruction']


In [2]:
# 骨骼连接数据
connections = [
    [0, 1], [1, 2], [2, 3], [0, 4], [4, 5],
    [5, 6], [0, 7], [7, 8], [8, 9], [9, 10],
    [8, 11], [11, 12], [12, 13], [8, 14], [14, 15], [15, 16]
]
# 点的标签


In [3]:
def calculate_angle_between_vectors(v1, v2):
    """Calculate the angle in degrees between two vectors."""
    unit_vector_1 = v1 / np.linalg.norm(v1)
    unit_vector_2 = v2 / np.linalg.norm(v2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    return np.degrees(angle)


# 添加角度显示到图形中
def add_angle_display(fig, pos1, pos2, pos3, label):

    vector1 = pos2 - pos1
    vector2 = pos2 - pos3
    angle = calculate_angle_between_vectors(vector1, vector2)
    angle_text = f"{angle:.2f}°"

    fig.add_trace(go.Scatter3d(
        x=[pos2[0]],  
        y=[pos2[2]],  # 适应您的轴设置
        z=[-pos2[1]],
        text=[angle_text],
        mode='text',
        textfont=dict(size=20),
        textposition="top center"
    ))

        # 计算中点  
    midpoint1 = (pos1 + pos2) / 2  
    midpoint2 = (pos2 + pos3) / 2  
    
    # 生成链接曲线  
    t = np.linspace(0, 1, 100)  
    curve_x = (1 - t) * midpoint1[0] + t * midpoint2[0]  
    curve_y = (1 - t) * midpoint1[2] + t * midpoint2[2]  # 适应您的轴设置  
    curve_z = -((1 - t) * midpoint1[1] + t * midpoint2[1])  
    
    fig.add_trace(go.Scatter3d(  
        x=curve_x,  
        y=curve_y,  
        z=curve_z,  
        mode='lines',  
        line=dict(color='green', width=2)  
    ))  

# 添加角度显示到图形中
def add_angle_displayV2(fig, pos1, pos2, vector2, label):
    vector1 = pos2 - pos1
    vector2 = vector2
    angle = calculate_angle_between_vectors(vector1, vector2)
    angle_text = f"{angle:.2f}°"
    fig.add_trace(go.Scatter3d(
        x=[pos2[0]],  
        y=[pos2[2]],  # 适应您的轴设置
        z=[-pos2[1]],
        text=[angle_text],
        mode='text',
        textfont=dict(size=20),
        textposition="top center"
    ))

def get_joint_position(data, index):  
    return data[index]  

In [4]:
# 初始化FigureWidget
# 初始化FigureWidget并设置尺寸  
# 初始化FigureWidget并设置尺寸（使用数字而不是字符串）  
fig = go.FigureWidget(  
    layout={  
        'width': 1000,  # 可以调整宽度，例如800, 1000等  
        'height': 800   # 可以调整高度，例如600, 800等  
    }  
)  

# 设置布局
fig.update_layout(scene=dict(
                    xaxis=dict(title='X'),
                    yaxis=dict(title='Y'),  # 确保y轴垂直方向
                    zaxis=dict(title='Z')
                ))


labels = ['Hips', 'RightThigh', 'RightShin', 'RightFoot', 'LeftThigh', 'LeftShin', 'LeftFoot', 'Spine', 'Neck', 'UpperNeck', 'TopHead', 'LeftShoulder', 'LeftArm', 'LeftHand', 'RightShoulder', 'RightArm', 'RightHand']
colors = [  
    (255, 87, 51),  
    (0, 191, 255),  
    (128, 0, 128),  
    (50, 205, 50),  
    (255, 215, 0),  
    (75, 0, 130)  
]

In [5]:
def display_3D(reconstruction_data, show_color, visible=True, index=0):
    frame_data = reconstruction_data[index]
    for connection in connections:
        x = [frame_data[connection[0], 0], frame_data[connection[1], 0]]
        y = [frame_data[connection[0], 2], frame_data[connection[1], 2]]  # 反转原z轴的值
        z = [-frame_data[connection[0], 1], -frame_data[connection[1], 1]]  # 使用原y轴的值
        fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='blue'), showlegend=False))
    # fig.add_trace(go.Scatter3d(x=frame_data[:, 0], y=frame_data[:, 2], z=-frame_data[:, 1], mode='markers', marker=dict(size=5, color='red')))

    # 示例：计算并显示大腿和小腿的角度
    right_thigh_pos = get_joint_position(frame_data, labels.index('RightThigh'))
    right_shin_pos = get_joint_position(frame_data, labels.index('RightShin'))
    right_foot_pos = get_joint_position(frame_data, labels.index('RightFoot'))
    
    add_angle_display(fig, right_thigh_pos, right_shin_pos, right_foot_pos, "Right Leg")


    # 示例：计算并显示大腿和小腿的角度
    Hips_pos = get_joint_position(frame_data, labels.index('Hips'))
    Head_pos = get_joint_position(frame_data, labels.index('TopHead'))
    vertical_vector =  np.array([0, 0, 1])  # 取默认的垂直向量 
    
    add_angle_displayV2(fig, Head_pos, Hips_pos, vertical_vector, "Body")

    # 对于每个标签添加一个点到图例
    for i, label in enumerate(labels):
        fig.add_trace(go.Scatter3d(
            x=[frame_data[i, 0]],
            y=[frame_data[i, 2]],
            z=[-frame_data[i, 1]],
            mode='markers',
            marker=dict(size=10, color=show_color),  # 指定颜色为红色            
            name=label,
        ))

# 为FigureWidget添加初始帧的数据
def init_figure():
    for index, npz in enumerate(npz_dict): 
        print(f"====> display {index} 3D, color {colors[index]}")
        display_3D(all_data[index], show_color=colors[index], visible=True, index=0)
    

# 滑动条的回调函数
def update_frame(change):
    for index, npz in enumerate(npz_dict): 
        print(f"====> display {index} 3D, color {colors[index]}")
        display_3D(all_data[index], show_color=colors[index], visible=True, index=change.new)


init_figure()
# 创建滑动条
slider = widgets.IntSlider(min=0, max=reconstruction_data.shape[0] - 1, step=1, value=0)
slider.observe(update_frame, names='value')
display(slider, fig)

====> display 0 3D, color (255, 87, 51)
====> display 1 3D, color (0, 191, 255)
====> display 2 3D, color (128, 0, 128)
====> display 3 3D, color (50, 205, 50)
====> display 4 3D, color (255, 215, 0)
====> display 5 3D, color (75, 0, 130)


IntSlider(value=0, max=182)

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter3d',
              'uid': '1cc90f0f-7bb0-47cd-82fc-eb94c49fb955',
              'x': [0.0, -0.08335307240486145],
              'y': [0.0, 0.06699547916650772],
              'z': [-0.0, 0.019231118261814117]},
             {'line': {'color': 'blue'},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter3d',
              'uid': 'b0d138ed-5c16-4749-aa6e-e8e33732d1df',
              'x': [-0.08335307240486145, 0.021182596683502197],
              'y': [0.06699547916650772, 0.1646597683429718],
              'z': [0.019231118261814117, -0.40460333228111267]},
             {'line': {'color': 'blue'},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter3d',
              'uid': '96940d0d-d8d3-4e3a-ac2d-ef30a08fc95a',
              'x': [0.0211825966835021

====> display 0 3D, color (255, 87, 51)
====> display 1 3D, color (0, 191, 255)
====> display 2 3D, color (128, 0, 128)
====> display 3 3D, color (50, 205, 50)
====> display 4 3D, color (255, 215, 0)
====> display 5 3D, color (75, 0, 130)
